In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
## Load the dataset
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Preprocess the data - Drop unnecessary columns/features
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Encoding categorical features - gender
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
# Encoding categorical features - geography - one-hot encoding
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
ohe_geography_encoded = ohe_geo.fit_transform(data[['Geography']])
ohe_geography_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
ohe_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
ohe_geography_encoded.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [12]:
geography_encoded_df=pd.DataFrame(ohe_geography_encoded.toarray(), columns=ohe_geo.get_feature_names_out(['Geography']))

In [13]:
geography_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
# Combining the encoded geography columns with the original data
data = pd.concat([data.drop('Geography', axis=1), geography_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
# Saving the label encoder and sclaer into pickle files

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('geography_encoded.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)

In [16]:
## Let's divide the dataset into dependent and independent features
X = data.drop('Exited', axis=1)
y = data['Exited']

## Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

In [18]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [19]:
# Saving the scaler into a pickle file
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Building the ANN

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [21]:
# Buiding the ANN model
model = Sequential()

# X_train.shape[0] = 8000, X_train.shape[1] = 11 --> We need the input_dim to be 11
# Adding the input layer and the first hidden layer
# The input_dim should be equal to the number of features in the training set
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1])) 
model.add(Dropout(0.2))

# Adding the second hidden layer
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))

model.summary()

/Users/saurabhbiswal/Documents/Udemy/E2E_Project_ANN/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [23]:
# Compliling the ANN
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
import datetime

## Setup the TensorBoard callback

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [25]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
# Early stopping callback
# TensorBoard callback
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fitting the ANN to the training set
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=100, callbacks=[tensorboard_callback, early_stopping])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8072 - loss: 0.4494 - val_accuracy: 0.8515 - val_loss: 0.3748
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.8429 - loss: 0.3858 - val_accuracy: 0.8560 - val_loss: 0.3508
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8519 - loss: 0.3631 - val_accuracy: 0.8530 - val_loss: 0.3483
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.8490 - loss: 0.3544 - val_accuracy: 0.8570 - val_loss: 0.3498
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - accuracy: 0.8587 - loss: 0.3522 - val_accuracy: 0.8560 - val_loss: 0.3440
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.8508 - loss: 0.3642 - val_accuracy: 0.8540 - val_loss: 0.3743
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - accuracy: 0.8508 - loss: 0.3575 - val_accuracy: 0.8585 - val_loss: 0.3407
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.8573 - loss: 0.348

In [26]:
model.save('model.h5')

In [27]:
# Load the tensorboard extension
%load_ext tensorboard

In [28]:
# To view the TensorBoard
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 31510), started 8:13:14 ago. (Use '!kill 31510' to kill it.)

# Predictions

In [45]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd

# Load model and preprocessing tools
model = load_model('model.h5')
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder = pickle.load(file)
with open('geography_encoded.pkl', 'rb') as file:
    ohe_geography = pickle.load(file)

# Sample input
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

# Transform Geography using OneHotEncoder
ip_geography_encoded = ohe_geography.transform([[input_data['Geography']]]).toarray()
geography_encoded_df = pd.DataFrame(
    ip_geography_encoded, 
    columns=ohe_geography.get_feature_names_out(['Geography'])
)

/Users/saurabhbiswal/Documents/Udemy/E2E_Project_ANN/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [46]:
geography_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [47]:
# Combine the input data with the encoded geography
input_df = pd.DataFrame([input_data])
input_df = pd.concat([input_df.drop('Geography', axis=1), geography_encoded_df], axis=1)

In [48]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [49]:
# label encode gender
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [40]:
## Scaling the input data
scaled_input_df = scaler.transform(input_df)
scaled_input_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [50]:
# Make prediction
prediction = model.predict(scaled_input_df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


array([[0.06140841]], dtype=float32)

In [51]:
# Prediction is a probability, we can convert it to binary
predicted_class = (prediction > 0.5).astype(int)
predicted_class
# Convert the predicted class to a human-readable format
if predicted_class[0][0] == 1:
    print("The customer is likely to exit.")
else:
    print("The customer is likely to stay.")

The customer is likely to stay.


In [52]:
## Integrating the model with Streamlit
# --> In another file (app.py)